<a href="https://colab.research.google.com/github/valentina-buoro/Postpartum-depression-prediction/blob/main/notebooks/Postpartum_EDA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [67]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

import warnings
warnings.filterwarnings('ignore')

In [68]:

PARENT_PATH = '/content/drive/MyDrive/'

data = pd.read_csv(PARENT_PATH + "Copy of THP_clean.csv")


data.head()


,newid,interviewer,uc,grandmother,employed_mo,income_mo,income_hh,genhealth,edu_mo,edu_fa,...,infantdev_1y,infantdev_1y_f,parentinputs_infancy,parentinputs_infancy_f,infantdev,infantdev_f,motherfinancial_7y,motherfinancial_7y_f,socialsupport,socialsupport_f
0,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,7.319728,...,-1.550027,-0.914190,0.845854,0.341055,-1.155602,-0.975243,NaN,NaN,1.496326,NaN
1,226.0,1.0,1,No,No,0.0,60.0,Very good,0.0,8.000000,...,0.919651,0.044339,2.737210,0.550107,0.587643,0.244921,-0.064279,-0.346433,2.569177,1.731267
2,222.0,6.0,1,Yes,No,0.0,200.0,Moderate,7.0,9.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,-0.521870,-0.295887,1.275779,NaN
3,3.0,1.0,1,No,No,0.0,150.0,Good,0.0,5.000000,...,1.589225,0.971888,0.634669,0.429139,0.326545,0.626777,0.521984,-0.163072,-0.521288,-0.880748
4,217.0,3.0,1,No,No,0.0,250.0,0,5.0,0.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,0.521984,-0.163072,1.275779,NaN


In [69]:
list(data.columns)

['newid',
 'interviewer',
 'uc',
 'grandmother',
 'employed_mo',
 'income_mo',
 'income_hh',
 'genhealth',
 'edu_mo',
 'edu_fa',
 'ideal_no_kids',
 'no_kids_over5_dead',
 'no_kids_1_5_dead',
 'no_kids_less1_dead',
 'mo_185',
 'mo_358',
 'mo_360',
 'c_wt',
 'c_ht',
 'ch_27',
 'ch_28',
 'ch_29',
 'ch_30',
 'ch_31',
 'ch_32',
 'ch_33',
 'ch_34',
 'ch_35',
 'ch_36',
 'mo_ht',
 'mo_bmi',
 'hamd_baseline',
 'bdq_baseline',
 'edu_mo_baseline',
 'edu_fa_baseline',
 'kids_no',
 'var29',
 'mspss_baseline',
 'depressed_6m',
 'hamd_6m',
 'bdq_6m',
 'mspss_6m',
 'ses_6m',
 'c_wt_6m',
 'c_ht_6m',
 'depressed_1y',
 'hamd_1y',
 'bdq_1y',
 'mspss_1y',
 'ses_1y',
 'pregnant_12m',
 'c_wt_1y',
 'c_ht_1y',
 'play_mo_1y',
 'play_fa_1y',
 'var611',
 'var618',
 'var619',
 'var620',
 'var621',
 'c_age_int',
 'vci',
 'vsi',
 'fri',
 'wmi',
 'psi',
 'fsiq',
 'sdq_emo',
 'sdq_cond',
 'sdq_hyper',
 'sdq_peer',
 'sdq_pro',
 'sdq_sum',
 'home_res',
 'home_mat',
 'home_emo',
 'home_learn',
 'home_enrich',
 'home_f_co

In [70]:
def convert_text_dataschema_to_csv(textfile:str):
 with open(textfile, 'r') as f:
    data = [re.match(r'la var\s+(\S+)\s+\"(.+)\"', line).groups()
            for line in f if line.startswith('la var')]

  # Convert to DataFrame and save
    df = pd.DataFrame(data, columns=['Variable Name', 'Label'])
    df.to_csv('schema.csv', index=False)

In [71]:
convert_text_dataschema_to_csv(PARENT_PATH + 'Copy of datasetschema.txt')

In [72]:
datashema_df = pd.read_csv('schema.csv')
datashema_df.head()

,Variable Name,Label
0,c_wt_6m,Child Weight (at 6 months)
1,c_wt_1y,Child Weight (at 1 year)
2,c_ht_6m,Child Height (at 6 months)
3,c_ht_1y,Child Height (at 1 year)
4,hamd_6m,Mother's Hamilton score (at 6 months)


In [73]:
list(datashema_df['Label'])

['Child Weight (at 6 months)',
 'Child Weight (at 1 year)',
 'Child Height (at 6 months)',
 'Child Height (at 1 year)',
 "Mother's Hamilton score (at 6 months)",
 "Mother's Hamilton score (at 1 year)",
 'Child Full Scale IQ (age 7)',
 'Child Weight (age 7)',
 'Child Height (age 7)',
 'Child Spence anxiety (age 7)',
 'Child SDQ (age 7)',
 'Home score (age 7)',
 'Treatment',
 'Girl $\\times$ Treat',
 'Depr $\\times$ Treat',
 'Depr',
 'Grandma $\\times$ Treat',
 'Low educ',
 'Low educ $\\times$ Treat',
 'Electricity',
 'Radio',
 'TV',
 'Refrigerator',
 'Bicycle',
 'Air con',
 'Washing machine',
 'Water pump',
 'Car',
 'Piped drinking water',
 'Flush toilet',
 'Brick/concrete walls',
 'No. member per room',
 'Has debt',
 'Husband supports in difficult situations (12mo)',
 'Treated',
 'Baseline depressed',
 "Mother's age",
 "Mother's age$^2$",
 'Nuclear family (baseline)',
 'Joint/extended family structure',
 'Child gender',
 "Mother's education (yrs)",
 "Father's education (yrs)",
 'Parity

In [77]:
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
data.head()

,newid,interviewer,uc,grandmother,employed_mo,income_mo,income_hh,genhealth,edu_mo,edu_fa,ideal_no_kids,no_kids_over5_dead,no_kids_1_5_dead,no_kids_less1_dead,mo_185,mo_358,mo_360,c_wt,c_ht,ch_27,ch_28,ch_29,ch_30,ch_31,ch_32,ch_33,ch_34,ch_35,ch_36,mo_ht,mo_bmi,hamd_baseline,bdq_baseline,edu_mo_baseline,edu_fa_baseline,kids_no,var29,mspss_baseline,depressed_6m,hamd_6m,bdq_6m,mspss_6m,ses_6m,c_wt_6m,c_ht_6m,depressed_1y,hamd_1y,bdq_1y,mspss_1y,ses_1y,pregnant_12m,c_wt_1y,c_ht_1y,play_mo_1y,play_fa_1y,var611,var618,var619,var620,var621,c_age_int,vci,vsi,fri,wmi,psi,fsiq,sdq_emo,sdq_cond,sdq_hyper,sdq_peer,sdq_pro,sdq_sum,home_res,home_mat,home_emo,home_learn,home_enrich,home_f_comp,home_f_inter,home_env,home,stroop,panic,separation,injury_fear,social_phobia,obc,gad,spence,three_groups,treat,attrit,dep_sample,Group,intervr_1,intervr_2,intervr_3,intervr_4,intervr_5,intervr_6,intervr_7,intervr_8,intervr_9,lhw_1,lhw_2,lhw_3,lhw_4,lhw_5,lhw_6,lhw_7,lhw_8,lhw_9,lhw_10,lhw_11,lhw_12,lhw_13,lhw_14,lhw_15,lhw_16,lhw_17,lhw_18,lhw_19,lhw_20,lhw_21,lhw_22,lhw_23,lhw_24,lhw_25,lhw_26,lhw_27,lhw_28,lhw_29,lhw_30,lhw_31,lhw_32,lhw_33,lhw_34,lhw_35,lhw_36,lhw_37,lhw_38,lhw_39,lhw_40,lhw_41,lhw_42,lhw_43,lhw_44,lhw_45,lhw_46,lhw_47,lhw_48,lhw_49,lhw_50,lhw_51,lhw_52,lhw_53,lhw_54,lhw_55,lhw_56,lhw_57,lhw_58,lhw_59,lhw_60,lhw_61,lhw_62,lhw_63,lhw_64,lhw_65,lhw_66,sample,mo_emp,gaf_baseline,var416,var599,HAZ_6,WAZ_6,HAZ_12,WAZ_12,month_int,month_int_sq,doi0,age,age_sq,age_baseline,no_kids_baseline,c_first_child,edu_parents,edu_lvl_mo_7y_1,edu_lvl_mo_7y_2,edu_lvl_mo_7y_3,edu_lvl_mo_7y_4,mspss_tot,depressed,recover_perm,recover_never,girl,girlXtreat,g4a,grandmother_baseline,employed_fa_baseline,occupation_fa_baseline,famstruct_baseline,grandmother_6m,grandmother_1y,first_child,employed_mo_baseline,edu_lvl_mo_1,edu_lvl_mo_2,edu_lvl_mo_3,ses_bl_flipped,electricity,radio,tv,fridge,bicycle,motorcycle,aircon,washingmachine,waterpump,cartruck,pipedwater,wellwithpump,wellwater,flushtoilet,anylatrine,brickwalls,enoughfoodmoney,metalroof,bestroof,wealth_baseline,age_baseline_sq,depXtreat,wealth_baseline_4sum,gender_4sum,THP_sample,attrit2,MIL,maternalgma,abortion,stillbirth,childdeath,motherdeath,refused,moved,zhaz,zbmi2,age_baselineXtreat,age_baseline_sqXtreat,employed_mo_baselineXtreat,mo_empXtreat,grandmother_baselineXtreat,MILXtreat,wealth_baselineXtreat,edu_lvl_mo_1Xtreat,edu_lvl_mo_2Xtreat,edu_lvl_mo_3Xtreat,edu_mo_baselineXtreat,edu_fa_baselineXtreat,kids_noXtreat,first_childXtreat,hamd_baselineXtreat,mspss_baselineXtreat,doi0Xtreat,no_kids_postt,age_kid1,femshare,notlast,kids_boy_share,p_hat,discussed,exclusivebf_6m,anybf_6m,diarhea_6m_flip,diarhea_1y_flip,ari_6m_flip,ari_1y_flip,urdu,math,private_sch,expect_sch,class_size,grandmaXtreat,toteduXtreat,control,motor,mo_491_2,mo_490_2,mo_485_2,mo_488_2,PPI,PPI_harsh,PPI_harsh4age,PPI_inconsistent,classroom,not_stunted,lteachers,lrooms,schoolqual_pca,home_obs,maritalscale,nonviolent,relationship_husb,relationship_inlaw,maritalscale_1y,relationship_husb_1y,relationship_inlaw_1y,nonviolent_1y,empowered,empowered_6m,basicneeds,enoughfood,ses,nodebt_7y,expend_food,expend_med,expend_educ,ln_expend_educ,ln_income_fa,ln_income_fa_6m,ln_income_fa_1y,notunwell,days_healthy,mo_wt_6m,notdepressed,scid_tot,impaired,notdep,notdep2011,notdep2012,notdep2013,dur_notdep,notdep1213,gaf_6m_2,gaf_1y_2,scid_tot_2,impaired_2,notdep1213_2,parentmoney,parenttime,parentstyle,parentstyle_f,cognindex,cognindex_f,healthindex,healthindex_f,emoindex,emoindex_f,incomeindex,incomeindex_f,motherhealthindex,motherhealthindex_f,motherfinancial,motherfinancial_f,fatherfinancial,fatherfinancial_f,relationshipindex,relationshipindex_f,relationshipindex_1y,relationshipindex_1y_f,relation_traj,relation_traj_f,grandmothers,grandmothers_f,fertility_vars,fertility_vars_f,childmort,childmort_f,depindex_0,depindex_0_f,depindex_6m,depindex_6m_f,depindex_1y,depindex_1y_f,depindex_7y,depindex_7y_f,mother_mh,mother_mh_f,depression_traj,depression_traj

In [78]:
pd.reset_option('display.max_columns')

In [80]:
data.shape

(1203, 394)

In [82]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1203 entries, 0 to 1202
Columns: 394 entries, newid to socialsupport_f
dtypes: float64(366), int64(11), object(17)
memory usage: 3.6+ MB


In [84]:
data.columns

Index(['newid', 'interviewer', 'uc', 'grandmother', 'employed_mo', 'income_mo',
       'income_hh', 'genhealth', 'edu_mo', 'edu_fa',
       ...
       'infantdev_1y', 'infantdev_1y_f', 'parentinputs_infancy',
       'parentinputs_infancy_f', 'infantdev', 'infantdev_f',
       'motherfinancial_7y', 'motherfinancial_7y_f', 'socialsupport',
       'socialsupport_f'],
      dtype='object', length=394)

In [85]:
def quick_column_overview(df):
    """Quick overview using pandas built-ins"""

    print("=== BASIC INFO ===")
    df.info(memory_usage='deep')  # Shows all columns with data types and null counts

    print("\n=== DESCRIPTIVE STATISTICS ===")
    print(df.describe(include='all'))  # Statistics for all columns

    print("\n=== DATA TYPES SUMMARY ===")
    print(df.dtypes.value_counts())  # Count of each data type

    print("\n=== MISSING DATA SUMMARY ===")
    missing_data = df.isnull().sum()
    missing_data = missing_data[missing_data > 0].sort_values(ascending=False)
    print(f"Columns with missing data: {len(missing_data)}")
    print(missing_data)

In [86]:
quick_column_overview(data)

=== BASIC INFO ===
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1203 entries, 0 to 1202
Columns: 394 entries, newid to socialsupport_f
dtypes: float64(366), int64(11), object(17)
memory usage: 4.4 MB

=== DESCRIPTIVE STATISTICS ===
             newid  interviewer           uc grandmother employed_mo  \
count   885.000000   885.000000  1203.000000         885         885   
unique         NaN          NaN          NaN           2           2   
top            NaN          NaN          NaN          No          No   
freq           NaN          NaN          NaN         538         781   
mean    446.225989     4.672316    20.358271         NaN         NaN   
std     256.597500     2.254451    11.440891         NaN         NaN   
min       1.000000     1.000000     1.000000         NaN         NaN   
25%     225.000000     3.000000    10.000000         NaN         NaN   
50%     447.000000     5.000000    20.000000         NaN         NaN   
75%     668.000000     6.000000    30.00000